In [1]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from qutip import *
import matplotlib.pyplot as plt
from visualisation_tools import *
from model_generator import *
from qsq_protocol import *
import datetime
import itertools
import random
plt.rcParams.update({
    "figure.figsize": (10, 6),
    "text.usetex": False, # Remove this line if your code doesn't run/ take too long
    "font.family": 'Times New Roman',
    "figure.dpi": 100,
    "font.size": 14
})

In [ ]:
#  Define the CSV file path
csv_file = 'uniform_data.csv'
generate_model_data(csv_file, 'uniform',3)

d:\Program Files\python\Lib\site-packages\qutip\core\data\expm.py:136: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  return Dense(scipy.linalg.sqrtm(matrix.as_ndarray()))


In [14]:
import numpy as np
from scipy.optimize import minimize, dual_annealing, basinhopping, differential_evolution
from qutip import rand_dm, rand_unitary

# Assuming you have compute_U, average_fidelity, decompose_U, plus_state, and minus_state defined

# Define the objective function to minimize
def objective_function(params, rho_exp, gate_exp, M_exp):
    # Unpack the parameters (theta1, theta2, theta3)
    theta1, theta2, theta3 = params
    
    # Compute the unitary operator U based on theta1, theta2, theta3
    U = compute_U(theta1, theta2, theta3)
    
    # Compute the objective function (average fidelity)
    return 1 - average_fidelity(U @ rho_exp @ U.dag(), U @ gate_exp @ U.dag(), U @ M_exp @ U.dag())

# Define optimization function
def optimize_method(method, objective_function, initial_guess, rho_exp, gate_exp, M_exp, bounds=None):
    if method == 'BFGS' or method == 'COBYLA':
        result = minimize(
            objective_function, 
            initial_guess, 
            bounds=bounds,  
            args=(rho_exp, gate_exp, M_exp),  
            method=method
        )
    elif method == 'differential_evolution':
        result = differential_evolution(
            objective_function, 
            bounds=bounds,
            args=(rho_exp, gate_exp, M_exp),
            maxiter=2000,
            popsize=30
        )
    elif method == 'basinhopping':
        result = basinhopping(
            objective_function, 
            initial_guess,  
            minimizer_kwargs={"method": "COBYLA", "args": (rho_exp, gate_exp, M_exp)},  
        )
    else:
        raise ValueError("Unknown optimization method.")
    return result

# Define bounds for optimization
bounds = [(-np.pi, np.pi), (0, np.pi/2), (-np.pi, np.pi)]


In [9]:
# Usage
data = load_model_data('uniform_data.csv')

# Access the data by variable name, for example:
print(data['models'])     # Access the models (now Qobjs)


[[Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.50689913-0.33394126j -0.79466301+0.00687156j]
 [ 0.35780753+0.70958458j -0.06473172+0.60355057j]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]]], [Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=False
Qobj data =
[[ 0.50689913-0.33394126j -0.79466301+0.00687156j]
 [ 0.35780753+0.70958458j -0.06473172+0.60355057j]], Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]]], [Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0.]
 [0. 0.]], Quant

In [ ]:
def average_fidelity_gauge(rho, gate, M, avg_fidelity=None):
    """
    Compute the gauge-corrected average fidelity.
    """
    if avg_fidelity is None:
        avg_fidelity = average_fidelity(rho,gate,M)
    
    eigenvalues, eigenstates = M.eigenstates()
    U_guess = eigenstates[0] @ plus_state.dag() - 1j * eigenstates[1] @ minus_state.dag()
    initial_guess = decompose_U(U_guess)
    bounds = [(-np.pi, np.pi), (0, np.pi/2), (-np.pi, np.pi)]
    result_C = optimize_method('basinhopping', objective_function, initial_guess, rho, gate, M, bounds)
    result_D = optimize_method('differential_evolution', objective_function, initial_guess, rho, gate, M, bounds)
    result = max(1-result_C.fun, 1-result_D.fun, avg_fidelity)
    
    return result


In [ ]:
def fill_model_gauge_fidelity(csv_file, flush_every=10):
    processed_rows = []

    with open(csv_file, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames

        for i, row in enumerate(reader, start=1):
            # Fill only if empty
            if not row.get('model_average_fidelity_gauge'):
                model_data = eval(row['model'])
                rho, gate, M = [Qobj(m) for m in model_data]
                avg_fidelity = float(row['model_average_fidelity'])

                row['model_average_fidelity_gauge'] = average_fidelity_gauge(
                    rho, gate, M, avg_fidelity
                )

            processed_rows.append(row)

            # Flush every N rows
            if i % flush_every == 0:
                _rewrite_csv(csv_file, fieldnames, processed_rows)

    # Final flush (for remainder)
    _rewrite_csv(csv_file, fieldnames, processed_rows)

def _rewrite_csv(csv_file, fieldnames, rows):
    tmp_file = csv_file + ".tmp"

    with open(tmp_file, mode='w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

    # Atomic replace (prevents corruption on crash)
    os.replace(tmp_file, csv_file)
